# Data preparation for calibration benchmarks

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from copy import deepcopy, copy
import os

from ctapipe.utils import get_dataset_path
from ctapipe.io import event_source
from ctapipe.calib import CameraCalibrator
from ctapipe.visualization import CameraDisplay
from ctapipe.io import HDF5TableWriter
from ctapipe.io.containers import Container, Field


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
from ctabench.dl1 import true_pe_cleaning, noise_calibrated_image, signal_calibrated_image

In [ ]:
class IntegrationContainer(Container):
    true_pe_sum = Field(float, 'true_pe_sum')
    true_pe_mean = Field(float, 'true_pe_mean')
    calibrated_signal_sum = Field(float, 'calibrated_signal_sum')
    calibrated_background_sum = Field(float, 'calibrated_background_sum')
    calibrated_signal_mean = Field(float, 'calibrated_signal_mean')
    calibrated_background_mean = Field(float, 'calibrated_signal_mean')
    

def difference_calibrated_pe(calibrated_event, tel_id, channel=0):
    """
    Return the difference between the calibrated signal and the true pe signal
    """
    pe_image = event.mc.tel[tel_id].photo_electron_image
    calibrated_image = event.dl1.tel[tel_id].image[channel]
    signal_pixels = true_pe_cleaning(pe_image)
    diff = pe_image[signal_pixels] - calibrated_image[signal_pixels]
    return diff.sum()

In [ ]:
# infile = get_dataset_path('gamma_test_large.simtel.gz')
infile = 'gamma_40deg_0deg_run102___cta-prod3-lapalma3-2147m-LaPalma_cone10.simtel.gz'
source = event_source(infile, max_events=None)

In [ ]:
integrators = {0: 'NeighbourPeakIntegrator',
               1: 'LocalPeakIntegrator',
               2: 'FullIntegrator'
              }

In [ ]:
int_container = IntegrationContainer()


output_filename = 'integration.h5'
if os.path.exists(output_filename):
    os.remove(output_filename)

for ii, integrator in integrators.items():
    print(integrator)
    
    with HDF5TableWriter(output_filename, group_name=integrator, mode='a') as h5_table:

        cal = CameraCalibrator(r1_product='HESSIOR1Calibrator',
                           extractor_product=integrator)

        for event in source:
            cal.calibrate(event)
            diff_sum = 0
            for tel_id in event.r0.tels_with_data:
                
                camera = event.inst.subarray.tel[tel_id].camera
                diff_sum += difference_calibrated_pe(event, tel_id)
                true_signal = event.mc.tel[tel_id].photo_electron_image
                image = event.dl1.tel[tel_id].image[0]
                signal = image[true_signal>0]
                background = image[true_signal==0]
                
                int_container.true_pe_sum = true_signal.sum()
                int_container.true_pe_mean = true_signal.mean()
                int_container.calibrated_signal_sum = signal.sum()
                int_container.calibrated_signal_mean = signal.mean()
                int_container.calibrated_background_sum = background.sum()
                int_container.calibrated_background_mean = background.mean()
                
                h5_table.write(camera.cam_id, [event.r0, event.mc, int_container])
